<a href="https://colab.research.google.com/github/EliRub1/Introduction-to-Cloud-Computing/blob/main/ItCC_Tut6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup

def fetch_page(url):
 response = requests.get(url)
 if response.status_code == 200:
  soup = BeautifulSoup(response.text, 'html.parser')
  return soup
 else:
  return None
#saves the text only from the given url

In [ ]:
import re
def index_words(soup):
  index = {}
  words = re.findall(r'\w+', soup.get_text())
  for word in words:
    word = word.lower()
    if word in index:
      index[word] += 1
    else:
      index[word] = 1
  return index

  #counts how many times any word is in the text

In [ ]:
def remove_stop_words(index):
  stop_words = {'a', 'an', 'the', 'and', 'or',     				'in', 'on', 'at'}
  for stop_word in stop_words:
    if stop_word in index:
      del index[stop_word]
  return index

  #removes the stopwords from the text

In [ ]:
from nltk.stem import PorterStemmer
def apply_stemming(index):
  stemmer = PorterStemmer()
  stemmed_index = {}
  for word, count in index.items():
    stemmed_word = stemmer.stem(word)
    if stemmed_word in stemmed_index:
      stemmed_index[stemmed_word] += count
    else:
      stemmed_index[stemmed_word] = count
  return stemmed_index

  #בודקים את מספר המופעים של מילים עם אותה המשמעות: example => plays,playing,played ==> play

In [ ]:
def search(query, index):
  query_words = re.findall(r'\w+', query.lower())
  results = {}
  for word in query_words:
    if word in index:
      results[word] = index[word]
  return results

  #checks if the searched word is in the query word(שאילתה) and saves the index num (לצורך הדוגמא ככה רואים אם יש מידע שאנו רוצים בדף..)

In [ ]:
def search_engine(url, query):
  soup = fetch_page(url)
  if soup is None:
     return None
  index = index_words(soup)
  index = remove_stop_words(index)
  index = apply_stemming(index)
  results = search(query, index)
  return results

  #searchs for the query in the given url

In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'bird'
results = search_engine(url, query)
print(results)
#counts how many time the word  bird is in the wiki page

In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'birds wings'
results = search_engine(url, query)
print(results)

In [ ]:
def search(query, index):
  stemmer = PorterStemmer()
  query_words = re.findall(r'\w+', query.lower())
  results = {}
  for word in query_words:
    word = stemmer.stem(word)
    if word in index:
      results[word] = index[word]
  return results

  #change the way the search is made by stemming the words and that counts it.

In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'birds wings'
results = search_engine(url, query)
print(results)
rank=1
for word, count in results.items():
   rank = rank*1/count
rank = 1-rank
print(rank)

#made a ranking system and gave the page the rank 0.99999 thats a high rank because the words birds+wings are shown a lot in the page

In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'collage students'
results = search_engine(url, query)
print(results)
rank=1
for word, count in results.items():
 rank = rank*1/count
rank = 1-rank
print(rank)
# the rank is 0 because there is no collage students mentioned in the page

In [ ]:
url = 'https://en.wikipedia.org/wiki/Bird'
query = 'owls'
results = search_engine(url, query)
print(results)
rank=1
for word, count in results.items():
 rank = rank*1/count
rank = 1-rank
print(rank)

#good rank because the word owl is in the page +-

In [ ]:
url = 'https://w3.braude.ac.il/?lang=en'
query = 'Industry'
results = search_engine(url, query)
print(results)
rank=1
for word, count in results.items():
 rank = rank*1/count
rank = 1-rank
print(rank)

#searching for the word industry in the braude collage site

In [ ]:
url = 'https://w3.braude.ac.il/?lang=en'
query = 'Braude collage'
results = search_engine(url, query)
print(results)
rank=1
for word, count in results.items():
 rank = rank*1/count
rank = 1-rank
print(rank)

#searching for the words braude collage in the braude collage site

In [ ]:
url = 'https://w3.braude.ac.il/?lang=en'
query = 'Galilee center'
results = search_engine(url, query)
print(results)
rank=1
for word, count in results.items():
 rank = rank*1/count
rank = 1-rank
print(rank)
#searching for the words Galilee center in the braude collage site

**engine for multiple pages**

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
class WikiSearchEngine:
  def __init__(self):
    """Initialize the search engine"""
    self.base_url = "https://en.wikipedia.org/w/api.php"

    self.pages = []
    self.word_locations = defaultdict(list) # word -> [(page_id, frequency), ...]
    self.stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to', 'for', 'of', 'with'}

  def fetch_wiki_pages(self, topic, num_pages=5):
    """Fetch Wikipedia pages for given topic"""
    search_params = {
      "action": "query",
      "format": "json",
      "list": "search",
      "srsearch": topic,
      "srlimit": num_pages
    }
    try:
      response = requests.get(self.base_url, params=search_params)
      search_results = response.json()['query']['search']

      for result in search_results:
        content_params = {
            "action": "query",
            "format": "json",
            "prop": "extracts|info",
            "pageids": result['pageid'],
            "inprop": "url",
            "explaintext": True
        }
        content_response = requests.get(self.base_url, params=content_params)
        page_data = content_response.json()['query']['pages'][str(result['pageid'])]
        self.pages.append({
          'id': result['pageid'],
          'title': page_data['title'],
          'url': page_data.get('fullurl', f"https://en.wikipedia.org/?curid={result['pageid']}"),
          #'url': page_data.get('fullurl', f"mqtt.org"),
          'content': page_data['extract']
        })
      print(f"Retrieved: {page_data['title']}")
      return True

    except Exception as e:
      print(f"Error fetching pages: {str(e)}")

  def build_index(self):
        """Build a simple word location index"""
        self.word_locations.clear()

        # Process each page
        for page in self.pages:
            # Get all words from content
            words = re.findall(r'\w+', page['content'].lower())

            # Count word frequencies
            word_counts = defaultdict(int)
            for word in words:
                if word not in self.stop_words:
                    word_counts[word] += 1

            # Add to index with page information
            for word, count in word_counts.items():
                self.word_locations[word].append((page['id'], count))

  def get_context(self, content, query_words, window=30):
        """Get a short snippet of text around the first matching word"""
        content_lower = content.lower()
        for word in query_words:
            index = content_lower.find(word)
            if index != -1:
                start = max(index - window, 0)
                end = min(index + window, len(content))
                return content[start:end].strip()
        return "No context found."

  def search(self, query, num_results=5):
        """Search pages using simple word frequency ranking.
        Ranks pages based on:1. Number of query words found in the page
        2. Total frequency of query words  """
        # Get query words
        query_words = [word.lower() for word in re.findall(r'\w+', query)
                    if word.lower() not in self.stop_words]
        if not query_words:
            return []

        # Calculate scores for each page
        page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})

        # For each query word
        for word in query_words:
            # Find pages containing this word
            for page_id, freq in self.word_locations.get(word, []):
                page_scores[page_id]['matches'] += 1
                page_scores[page_id]['total_freq'] += freq


        # Convert to list and sort
        ranked_results = [
            (page_id, scores['matches'], scores['total_freq'])
            for page_id, scores in page_scores.items()
        ]
        # Sort by number of matching words first, then by total frequency
        ranked_results.sort(key=lambda x: (x[1], x[2]), reverse=True)
        # Format results
        results = []
        for page_id, matches, total_freq in ranked_results[:num_results]:
            page = next(p for p in self.pages if p['id'] == page_id)
            # Find the first matching word context
            context = self.get_context(page['content'], query_words)
            results.append({
                'title': page['title'],
                'url': page['url'],
                'matching_words': matches,
                'total_frequency': total_freq,
                'context': context
            })
        return results


In [ ]:
def fetch_wiki_pages(self, topic, num_pages=5):
  """Fetch Wikipedia pages for given topic"""
  search_params = {
  "action": "query",
  "format": "json",
  "list": "search",
  "srsearch": topic,
  "srlimit": num_pages
  }
  try:
      response = requests.get(self.base_url, params=search_params)
      search_results = response.json()['query']['search']

      for result in search_results:
        content_params = {
            "action": "query",
            "format": "json",
            "prop": "extracts|info",
            "pageids": result['pageid'],
            "inprop": "url",
            "explaintext": True
        }
        content_response = requests.get(self.base_url, params=content_params)
        page_data = content_response.json()['query']['pages'][str(result['pageid'])]
        self.pages.append({
          'id': result['pageid'],
          'title': page_data['title'],
          'url': page_data.get('fullurl', f"https://en.wikipedia.org/?curid={result['pageid']}"),
          'content': page_data['extract']
        })
        print(f"Retrieved: {page_data['title']}")
      return True

  except Exception as e:
    print(f"Error fetching pages: {str(e)}")


In [ ]:
    def build_index(self):
        """Build a simple word location index"""
        self.word_locations.clear()

        # Process each page
        for page in self.pages:
            # Get all words from content
            words = re.findall(r'\w+', page['content'].lower())

            # Count word frequencies
            word_counts = defaultdict(int)
            for word in words:
                if word not in self.stop_words:
                    word_counts[word] += 1

            # Add to index with page information
            for word, count in word_counts.items():
                self.word_locations[word].append((page['id'], count))

    def search(self, query, num_results=5):
        """Search pages using simple word frequency ranking.
        Ranks pages based on:1. Number of query words found in the page
        2. Total frequency of query words  """
        # Get query words
        query_words = [word.lower() for word in re.findall(r'\w+', query)
                      if word.lower() not in self.stop_words]
        if not query_words:
            return []

        # Calculate scores for each page
        page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})

        # For each query word
        for word in query_words:
            # Find pages containing this word
            for page_id, freq in self.word_locations.get(word, []):
                page_scores[page_id]['matches'] += 1
                page_scores[page_id]['total_freq'] += freq


        # Convert to list and sort
        ranked_results = [
            (page_id, scores['matches'], scores['total_freq'])
            for page_id, scores in page_scores.items()
        ]
        # Sort by number of matching words first, then by total frequency
        ranked_results.sort(key=lambda x: (x[1], x[2]), reverse=True)
        # Format results
        results = []
        for page_id, matches, total_freq in ranked_results[:num_results]:
            page = next(p for p in self.pages if p['id'] == page_id)
            # Find the first matching word context
            context = self.get_context(page['content'], query_words)
            results.append({
                'title': page['title'],
                'url': page['url'],
                'matching_words': matches,
                'total_frequency': total_freq,
                'context': context
            })
        return results

In [ ]:
# יצירת מופע של המנוע
engine = WikiSearchEngine()

# שליפת עמודים מוויקיפדיה (לדוגמה: נושא Python Programming)
engine.fetch_wiki_pages("Python programming", num_pages=5)

# בניית אינדקס מילים
engine.build_index()

# ביצוע חיפוש
results = engine.search("dynamic typing language")

# הצגת התוצאות
for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(f"Title: {result['title']}")
    print(f"URL: {result['url']}")
    print(f"Matching Words: {result['matching_words']}")
    print(f"Total Frequency: {result['total_frequency']}")
    print(f"Context: {result['context']}")
    print("-" * 60)


In [ ]:
# יצירת מופע של המנוע
engine = WikiSearchEngine()

# שליפת עמודים מוויקיפדיה (לדוגמה: נושא Python Programming)
engine.fetch_wiki_pages("software", num_pages=5)

# בניית אינדקס מילים
engine.build_index()

# ביצוע חיפוש
results = engine.search("dynamic typing language")

# הצגת התוצאות
for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(f"Title: {result['title']}")
    print(f"URL: {result['url']}")
    print(f"Matching Words: {result['matching_words']}")
    print(f"Total Frequency: {result['total_frequency']}")
    print(f"Context: {result['context']}")
    print("-" * 60)


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict

class MQTTSimpleSearch:
    def __init__(self):
        self.url = "https://mqtt.org/"
        self.text = ""
        self.word_index = defaultdict(int)
        self.stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to', 'for', 'of', 'with'}

    def fetch_site(self):
        try:
            response = requests.get(self.url)
            soup = BeautifulSoup(response.content, 'html.parser')
            self.text = soup.get_text().lower()
            print("Site content fetched.")
        except Exception as e:
            print("Error fetching site:", str(e))

    def build_index(self):
        words = re.findall(r'\w+', self.text)
        for word in words:
            if word not in self.stop_words:
                self.word_index[word] += 1
        print("Index built with", len(self.word_index), "unique words.")

    def search(self, query):
        query_words = [word.lower() for word in re.findall(r'\w+', query)
                       if word.lower() not in self.stop_words]
        results = {word: self.word_index[word] for word in query_words if word in self.word_index}
        return results


In [ ]:
engine = MQTTSimpleSearch()
engine.fetch_site()
engine.build_index()
print(engine.search("connect protocol broker"))


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import nltk
nltk.download('punkt')

# רשימת stop words בסיסית (ניתן לשנות בהתאם לצורך)
stop_words = {
    'a', 'an', 'the', 'in', 'on', 'at', 'for', 'to', 'from', 'is', 'are',
    'of', 'and', 'or', 'by', 'with', 'that', 'this', 'it', 'as', 'be', 'was', 'were'
}

# פונקציה לניקוי תוכן הדף
def clean_text(text):
    words = re.findall(r'\w+', text.lower())
    return [word for word in words if word not in stop_words]

# פונקציה לשליפת הקישורים הראשיים מתוך אתר mqtt.org
def get_mqtt_links(base_url="https://mqtt.org/"):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.find_all('a', href=True)
    valid_links = set()
    for link in links:
        href = link['href']
        if href.startswith('/') or href.startswith(base_url):
            if not href.startswith('mailto:'):
                full_url = href if href.startswith('http') else base_url.rstrip('/') + '/' + href.lstrip('/')
                valid_links.add(full_url)
    return list(valid_links)

# בניית האינדקס
def build_index(links):
    index = defaultdict(set)
    for i, url in enumerate(links):
        try:
            resp = requests.get(url, timeout=5)
            soup = BeautifulSoup(resp.text, 'html.parser')
            text = soup.get_text()
            words = clean_text(text)
            for word in words:
                index[word].add(i)  # ניתן גם לשמור את ה-URL המלא במקום מספר מזהה
        except Exception as e:
            print(f"Failed to fetch {url}: {e}")
    return index


In [ ]:
# שלב ראשון: שליפת קישורים
mqtt_links = get_mqtt_links()

# שלב שני: בניית האינדקס
mqtt_index = build_index(mqtt_links)

# שלב שלישי: הדפסת אינדקס לדוגמה (20 מונחים ראשונים)
for i, (term, doc_ids) in enumerate(mqtt_index.items()):
    print({"term": term, "DocIDs": list(doc_ids)})
    if i > 20:
        break
